In [286]:
import sys
sys.path.append('..')
import processing
import configs.common as cc

loader = processing.DatasetLoader('E:\\GitHub\\dataset\\np_dataset')
train_dataloader, test_dataloader = loader.get_dataloaders()
# random_sample = loader.get_random_sample('train')
# random_sample
for src, trg, meta in train_dataloader:
    break
# src, trg, meta = random_sample

In [287]:
from train import new_model
import torch
type = 'xlstm'
model_name = 'loss_0.16_time_2025-05-12-18-38-40.pth'
model = new_model(type)
model.load_state_dict(torch.load(f'../pretrained/{type}/{model_name}'))
model.to('cuda')

xLSTM(
  (token_embedding_table): Embedding(963, 256)
  (metadata_embedding_table): Embedding(154, 256)
  (layers): ModuleList(
    (0): sLSTMblock(
      (ln): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (conv): CausalConv1D(
        (conv): Conv1d(256, 256, kernel_size=(32,), stride=(1,), padding=(31,))
      )
      (drop): Dropout(p=0.01, inplace=False)
      (i_gate): BlockDiagonal(
        (blocks): ModuleList(
          (0-7): 8 x Linear(in_features=256, out_features=32, bias=True)
        )
      )
      (f_gate): BlockDiagonal(
        (blocks): ModuleList(
          (0-7): 8 x Linear(in_features=256, out_features=32, bias=True)
        )
      )
      (o_gate): BlockDiagonal(
        (blocks): ModuleList(
          (0-7): 8 x Linear(in_features=256, out_features=32, bias=True)
        )
      )
      (z_gate): BlockDiagonal(
        (blocks): ModuleList(
          (0-7): 8 x Linear(in_features=256, out_features=32, bias=True)
        )
      )
      (ri_gate):

In [297]:
import train
def generate(model, context_len, token_ids, meta_ids, num_tokens=1000, device='cpu'):
    model.eval()
    generated = token_ids.detach().cpu().numpy().tolist()[0]

    with torch.no_grad():
            for _ in range(num_tokens):
                print(_, end="\r")
                logits = model(token_ids, meta_ids)  # (1, seq_len, vocab_size)
                filtered_logits = train.filtered_logit(token_ids, logits)
                logits_last = filtered_logits[:, -1, :]       # (1, vocab_size)
                most_likely_token = logits_last[0].argmax(-1)
                num_occurrences = (token_ids[0][-30:] == most_likely_token).sum().item()
                logits_last[0][most_likely_token] -= 1 * num_occurrences
                if most_likely_token < cc.start_idx["dyn"]:
                    for idx in range(128):
                        num_occurrences = (token_ids[0][-30:] == idx).sum().item()
                        logits_last[0][idx] -= 2 * num_occurrences
                probs = torch.softmax(logits_last, dim=-1)  # (1, vocab_size)

                # Get top-5
                topk_probs, topk_indices = torch.topk(probs, 3, dim=-1)  # Each: shape (1, 5)
                if most_likely_token < 128:
                    print(logits_last[0][most_likely_token])
                    print(topk_indices)
                # Normalize to sum to 1
                topk_probs = topk_probs / topk_probs.sum(dim=-1, keepdim=True)

                # Sample
                next_token_idx = torch.multinomial(topk_probs, num_samples=1)  # (1, 1), value in [0..4]
                next_token = topk_indices.gather(1, next_token_idx)

                generated.append(next_token.item())

                token_ids = torch.cat([token_ids, next_token], dim=1)
                token_ids = token_ids[:, -context_len:]

            # If you want metadata, sample or set as zeros:
            # e.g., meta_ids = torch.cat([meta_ids, new_meta], dim=1)[:, -context_len:]

    return generated

new_seq = generate(model, cc.config.values.block_len, src[0].unsqueeze(0), meta[0].unsqueeze(0), num_tokens=80, device='cuda')

tensor(10.3179, device='cuda:0')
tensor([[57, 52, 77]], device='cuda:0')
tensor(10.3356, device='cuda:0')
tensor([[57, 75, 55]], device='cuda:0')
tensor(7.1950, device='cuda:0')
tensor([[75, 55, 77]], device='cuda:0')
tensor(7.2664, device='cuda:0')
tensor([[75, 79, 55]], device='cuda:0')
tensor(7.0935, device='cuda:0')
tensor([[55, 72, 62]], device='cuda:0')
tensor(7.5046, device='cuda:0')
tensor([[79, 76, 55]], device='cuda:0')
tensor(10.1704, device='cuda:0')
tensor([[52, 76, 62]], device='cuda:0')
tensor(9.8979, device='cuda:0')
tensor([[52, 57, 76]], device='cuda:0')
tensor(10.2337, device='cuda:0')
tensor([[57, 76, 55]], device='cuda:0')
tensor(6.7142, device='cuda:0')
tensor([[75, 43, 55]], device='cuda:0')
tensor(7.1923, device='cuda:0')
tensor([[55, 47, 76]], device='cuda:0')
tensor(6.8988, device='cuda:0')
tensor([[41, 43, 59]], device='cuda:0')
tensor(6.9048, device='cuda:0')
tensor([[62, 79, 76]], device='cuda:0')
tensor(6.9097, device='cuda:0')
tensor([[52, 77, 79]], devic

In [298]:
decoded_notes_old = processing.decode(src[0])
processing.note_to_midi(decoded_notes_old, "comparison.mid")

decoded_notes_new = processing.decode(new_seq[-80:])
processing.note_to_midi(decoded_notes_new, "generated.mid")

In [299]:
decoded_notes_new

[MIDI_note(pitch=51, time_start=0.42857142857142855, time_end=0.9285714285714286, dynamic=97, channel=48, tempo=210.0),
 MIDI_note(pitch=56, time_start=0.8571428571428571, time_end=0.9047619047619047, dynamic=99, channel=26, tempo=248.0),
 MIDI_note(pitch=76, time_start=0.8571428571428571, time_end=0.8974654377880183, dynamic=97, channel=48, tempo=210.0),
 MIDI_note(pitch=74, time_start=0.8571428571428571, time_end=0.8928571428571428, dynamic=115, channel=25, tempo=129.0),
 MIDI_note(pitch=71, time_start=0.8571428571428571, time_end=1.6710963455149501, dynamic=97, channel=0, tempo=248.0),
 MIDI_note(pitch=78, time_start=0.8571428571428571, time_end=1.3006912442396312, dynamic=94, channel=25, tempo=248.0),
 MIDI_note(pitch=61, time_start=0.8571428571428571, time_end=1.2805299539170507, dynamic=115, channel=33, tempo=248.0),
 MIDI_note(pitch=51, time_start=1.2200460829493087, time_end=1.2704493087557602, dynamic=97, channel=25, tempo=248.0),
 MIDI_note(pitch=56, time_start=1.220046082949

In [99]:
idx = 2
input = src[idx].unsqueeze(0)

In [167]:
logits = model(input, meta[idx].unsqueeze(0))
filtered_logits = train.filtered_logit(input, logits)
                
input = filtered_logits[0].argmax(-1).unsqueeze(0)
input

tensor([[ 58, 238, 299, 961, 473,  58, 239, 334, 961, 473,  51, 239, 334, 961,
         491,  27, 252, 355, 961, 482,  63, 185, 295, 360, 961, 484,  55, 248,
         355, 357, 961, 484,  58, 226, 355, 367, 961, 484,  63, 244, 355, 961,
         482,  55, 240, 295, 374, 961, 482,  58, 220, 295, 357, 961, 482,  63,
         179, 295, 360, 961, 482,  55, 218, 320, 395, 961, 484,  63, 235, 341,
         357, 961, 473,  70, 239, 330, 961, 473,  75, 239, 327, 961, 484,  55,
         237, 341, 961, 473,  67, 239, 327, 961, 484,  58, 246, 337, 961, 482,
          58, 219, 316, 961, 530,  63, 252, 281, 961, 482,  63, 219, 316, 961,
         530,  63, 252, 295, 399, 961, 491,  27, 252, 292, 357, 961, 482,  63,
         195, 278, 378, 961, 482,  58, 173, 274, 357, 961, 482,  55, 215, 274,
         961, 484,  38, 239, 355, 961, 484,  53, 241, 355, 371, 961, 482,  53,
         195, 299, 364, 961, 530,  62, 252, 334, 357, 961, 484,  50, 247, 348,
         961, 491,  26, 252, 355, 961, 482,  58, 229

In [169]:
filtered_logits[0][0][:128]

tensor([ 0.0000,  6.3134,  6.9035,  6.0828,  5.9433,  7.1644,  5.7620,  6.5757,
         6.4643,  6.3919,  6.1867,  6.8310,  6.3284,  6.4244,  6.5372,  6.4623,
         8.7010,  4.6767,  7.4200,  6.7567,  7.9998,  8.1419,  9.5407,  5.6227,
         6.7564,  4.8045,  3.8351, 10.1992,  7.5654,  8.9391,  5.8445,  7.6321,
         6.8122,  8.1876,  7.4191,  5.6611,  5.7854,  5.1118,  7.0683,  6.2696,
         8.7257,  7.1330,  5.1107,  5.9827,  5.5587, 10.4906,  8.5093,  5.9634,
         8.5304,  5.3487,  6.7490,  7.1678,  6.1069,  7.6541, 10.4402,  9.7946,
         8.7629,  6.1342, 20.6024,  4.4450,  8.1050,  9.6003,  8.3159,  5.5958,
         6.3124,  6.3709,  5.4957,  5.2022,  5.1336,  7.5839,  9.9579,  7.8871,
         5.0124,  9.3024,  7.5886,  6.8797,  6.0038,  5.4065,  8.6481,  8.2093,
         7.5058,  8.4233,  5.0589,  7.3166,  4.6714,  7.5599, 11.5220,  7.1528,
         5.1387,  8.8976,  6.4618,  8.3402,  7.3777,  5.5708,  6.6366,  5.1106,
         6.0629,  7.6522,  7.5989,  5.74

In [53]:
filtered_output[-1][-1].argmax(-1)
print(filtered_output[-1][-1][350:370])
print(filtered_output[-1][-1][950:970])

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 12.4743,
         6.8243,  6.7807, 11.2046,  8.2917,  8.2001,  9.5565,  8.2000,  8.4255,
         8.5841,  9.2970,  8.3535,  9.2126], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([ 6.8611,  5.2008,  5.6286,  7.7482,  7.3960,  6.3381,  6.2766,  5.8202,
         9.0022,  6.0999,  5.8930, 13.3240,  5.5961], device='cuda:0',
       grad_fn=<SliceBackward0>)


In [3]:
import torch
import torch.nn.functional as F

def make_distributions():
    # Prepare output tensor
    block_len = cc.config.values.block_len
    device = cc.config.values.device
    vocab_size = cc.vocab_size
    distributions = torch.zeros(6, vocab_size, device=device)

    # For each token index, fill in regions with 1 as per your logic.
    start = [cc.start_idx["pitch"],
             cc.start_idx["dyn"],
             cc.start_idx["length"],
             cc.start_idx["time"],
             cc.start_idx["channel"],
             cc.start_idx["tempo"]]
    end   = [cc.start_idx["dyn"] - 1,
             cc.start_idx["length"] - 1,
             cc.start_idx["time"] - 1,
             cc.start_idx["channel"] - 1,
             cc.start_idx["tempo"] - 1,
             cc.vocab_size]   # block_len implies : to the end

    for token in range(6):
        if token == 0:
            distributions[token, start[1]:end[1]] = 1
        if token == 1:
            distributions[token, start[2]:end[2]] = 1
        if token == 2:
            distributions[token, start[3]:end[3]] = 1
            distributions[token, start[5]:end[5]] = 1
        if token == 3:
            distributions[token, start[5]:end[5]] = 1
        if token == 4:
            distributions[token, start[0]:end[0]] = 1
        if token == 5:
            distributions[token, start[4]:end[4]] = 1

    return distributions


def pick_distributions_by_prev_token(
        input_tokens: torch.Tensor
    ) -> torch.Tensor:
    boundaries = [
                cc.start_idx['dyn'] - 1,
                cc.start_idx['length'] - 1,
                cc.start_idx['time'] - 1,
                cc.start_idx['channel'] - 1,
                cc.start_idx['tempo'] - 1]
        
    bins = torch.tensor(boundaries, device=input_tokens.device)
        # Each token is assigned to a bucket 0..len(boundaries)
        # For example, with bins=[10,20,30]: 
        #   token <10 → 0, 10<=token<20 → 1, 20<=token<30 → 2, 30<=token → 3
    buckets = torch.bucketize(input_tokens, bins, right=False)
    distributions = make_distributions()

    # Ensure buckets is long (int64) and on the same device as distributions
    buckets = buckets.long().to(distributions.device)

    # Now use advanced indexing to get values
    output = distributions[buckets]  # shape: [6, 963]

    return output

def cross_entropy_loss(input, output, target, reduction='mean'):
    weights = pick_distributions_by_prev_token(input)
    log_probs = F.log_softmax(output, dim=1)
    loss = log_probs * weights 
    loss = -log_probs.gather(1, target.unsqueeze(1)).squeeze(1)
    if reduction == 'mean':
        return loss.mean()
    elif reduction == 'sum':
        return loss.sum()
    else: # 'none'
        return loss

In [ ]:
def my_custom_loss(pred, target):
    # pred: (N, C), target: (N,)
    expected = pred.argmax(-1)
    probs = torch.log_softmax(pred, dim=1)
    loss = -probs[torch.arange(pred.shape[0]), target].mean()
    return loss
model.to("cuda")
output = model(src, meta)
output = output.reshape(-1, cc.vocab_size)
trg = trg.view(-1)
my_custom_loss(output, trg)

tensor(0.0048, device='cuda:0', grad_fn=<NegBackward0>)

In [7]:
output[-1][800:820]

tensor([-0.0704,  1.4107,  1.1998,  2.6456,  3.4005,  1.2786,  1.5597, -0.3986,
         0.5784,  0.5274,  2.8592,  0.4845, -1.1300,  0.1472,  0.6268,  1.2091,
         1.2610,  0.5960,  2.9577,  1.9989], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [29]:
max(output[-1])

tensor(9.0525, device='cuda:0', grad_fn=<UnbindBackward0>)

In [26]:
output[-1][350:360]

tensor([-0.8828, -1.6690, -2.4971, -4.0834, -2.2464, -3.8021, -1.7690,  9.0525,
         6.0496,  7.2018], device='cuda:0', grad_fn=<SliceBackward0>)

In [65]:
trg[-513]

tensor(43, device='cuda:0')

In [64]:
src[7][0]

tensor(883, device='cuda:0')

In [66]:
output[-513][30:50]

tensor([7.6896, 7.8589, 7.1193, 8.7035, 8.3538, 9.9633, 8.1413, 9.2453, 9.2601,
        8.3956, 8.1151, 9.1026, 8.0156, 7.8010, 8.5638, 8.9311, 7.8991, 7.7118,
        9.2435, 6.8082], device='cuda:0', grad_fn=<SliceBackward0>)